ДИСКЛЕЙМЕР! я бы не нажимала на запустить весь ноутбук, потому что там есть один нелинейный кусочек, лучше посмотрите сперва

# Домашнее задание 2. Классификация изображений.

В этом задании потребуется обучить классификатор изображений. Будем работать с датасетом, название которого раскрывать не будем. Можете посмотреть самостоятельно на картинки, которые в есть датасете. В нём 200 классов и около 5 тысяч картинок на каждый класс. Классы пронумерованы, как нетрудно догадаться, от 0 до 199. Скачать датасет можно вот [тут](https://yadi.sk/d/BNR41Vu3y0c7qA).

Структура датасета простая -- есть директории train/ и val/, в которых лежат обучающие и валидационные данные. В train/ и val/ лежат директориии, соответствующие классам изображений, в которых лежат, собственно, сами изображения.
 
__Задание__. Необходимо выполнить любое из двух заданий

1) Добейтесь accuracy **на валидации не менее 0.44**. В этом задании **запрещено** пользоваться предобученными моделями и ресайзом картинок. 

2) Добейтесь accuracy **на валидации не менее 0.84**. В этом задании делать ресайз и использовать претрейн можно. 

Напишите краткий отчёт о проделанных экспериментах. Что сработало и что не сработало? Почему вы решили, сделать так, а не иначе? Обязательно указывайте ссылки на чужой код, если вы его используете. Обязательно ссылайтесь на статьи / блогпосты / вопросы на stackoverflow / видосы от ютуберов-машинлернеров / курсы / подсказки от Дяди Васи и прочие дополнительные материалы, если вы их используете. 

Ваш код обязательно должен проходить все `assert`'ы ниже.

Необходимо написать функции `train_one_epoch`, `train` и `predict` по шаблонам ниже (во многом повторяют примеры с семинаров).Обратите особое внимание на функцию `predict`: она должна возвращать список лоссов по всем объектам даталоадера, список предсказанных классов для каждого объекта из даталоалера и список настоящих классов для каждого объекта в даталоадере (и именно в таком порядке).

__Использовать внешние данные для обучения строго запрещено в обоих заданиях. Также запрещено обучаться на валидационной выборке__.


__Критерии оценки__: Оценка вычисляется по простой формуле: `min(10, 10 * Ваша accuracy / 0.44)` для первого задания и `min(10, 10 * (Ваша accuracy - 0.5) / 0.34)` для второго. Оценка округляется до десятых по арифметическим правилам. Если вы выполнили оба задания, то берется максимум из двух оценок.

__Бонус__. Вы получаете 5 бонусных баллов если справляетесь с обоими заданиями на 10 баллов (итого 15 баллов). В противном случае выставляется максимальная из двух оценок и ваш бонус равен нулю.

__Советы и указания__:
 - Наверняка вам потребуется много гуглить о классификации и о том, как заставить её работать. Это нормально, все гуглят. Но не забывайте, что нужно быть готовым за скатанный код отвечать :)
 - Используйте аугментации. Для этого пользуйтесь модулем `torchvision.transforms` или библиотекой [albumentations](https://github.com/albumentations-team/albumentations)
 - Можно обучать с нуля или файнтюнить (в зависимости от задания) модели из `torchvision`.
 - Рекомендуем написать вам сначала класс-датасет (или воспользоваться классом `ImageFolder`), который возвращает картинки и соответствующие им классы, а затем функции для трейна по шаблонам ниже. Однако делать это мы не заставляем. Если вам так неудобно, то можете писать код в удобном стиле. Однако учтите, что чрезмерное изменение нижеперечисленных шаблонов увеличит количество вопросов к вашему коду и повысит вероятность вызова на защиту :)
 - Валидируйте. Трекайте ошибки как можно раньше, чтобы не тратить время впустую.
 - Чтобы быстро отладить код, пробуйте обучаться на маленькой части датасета (скажем, 5-10 картинок просто чтобы убедиться что код запускается). Когда вы поняли, что смогли всё отдебажить, переходите обучению по всему датасету
 - На каждый запуск делайте ровно одно изменение в модели/аугментации/оптимайзере, чтобы понять, что и как влияет на результат.
 - Фиксируйте random seed.
 - Начинайте с простых моделей и постепенно переходите к сложным. Обучение лёгких моделей экономит много времени.
 - Ставьте расписание на learning rate. Уменьшайте его, когда лосс на валидации перестаёт убывать.
 - Советуем использовать GPU. Если у вас его нет, используйте google colab. Если вам неудобно его использовать на постоянной основе, напишите и отладьте весь код локально на CPU, а затем запустите уже написанный ноутбук в колабе. Авторское решение задания достигает требуемой точности в колабе за 15 минут обучения.
 
Good luck & have fun! :)

In [1]:
import numpy as np
import torch
import torchvision
from torch import nn
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
from torchvision import models
import torchvision.transforms as transforms
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import sys
from torch import optim
import random

In [2]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    
set_random_seed(13)

In [28]:
!pip install wandb --upgrade --quiet
import wandb
wandb.login()
wandb.init(project="trial_and_error")



wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [4]:
!wget https://www.dropbox.com/s/33l8lp62rmvtx40/dataset.zip?dl=0
!unzip /content/dataset.zip?dl=0;
pass

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: dataset/dataset/val/class_101/00048.jpg  
  inflating: dataset/dataset/val/class_101/00049.jpg  
   creating: dataset/dataset/val/class_102/
  inflating: dataset/dataset/val/class_102/00000.jpg  
  inflating: dataset/dataset/val/class_102/00001.jpg  
  inflating: dataset/dataset/val/class_102/00002.jpg  
  inflating: dataset/dataset/val/class_102/00003.jpg  
  inflating: dataset/dataset/val/class_102/00004.jpg  
  inflating: dataset/dataset/val/class_102/00005.jpg  
  inflating: dataset/dataset/val/class_102/00006.jpg  
  inflating: dataset/dataset/val/class_102/00007.jpg  
  inflating: dataset/dataset/val/class_102/00008.jpg  
  inflating: dataset/dataset/val/class_102/00009.jpg  
  inflating: dataset/dataset/val/class_102/00010.jpg  
  inflating: dataset/dataset/val/class_102/00011.jpg  
  inflating: dataset/dataset/val/class_102/00012.jpg  
  inflating: dataset/dataset/val/class_102/00013.jpg  
  inflat

### Подготовка данных

In [5]:
train_transform = transforms.Compose([
    transforms.ToTensor()])
train_dataset = torchvision.datasets.ImageFolder("./dataset/dataset/train", transform=train_transform)
loader =  torch.utils.data.DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=True)

In [6]:
for image, _ in loader:
  mean = image.mean(dim = (0, 2, 3))
  std = image.std(dim = (0, 2, 3))

In [7]:
print('mean: ', mean, '\n std: ', std)

mean:  tensor([0.4802, 0.4481, 0.3975]) 
 std:  tensor([0.2770, 0.2691, 0.2821])


потом я подумала, что глупо как-то std считать по всему датасету, если потом нормализуем по батчам - по батчам std среднее будет меньше и, при применении, корректнее. среднее же вообще никак не измениться так что париться и пересчитывать его не будем

In [8]:
train_transform = transforms.Compose([
    transforms.ToTensor()])
train_dataset = torchvision.datasets.ImageFolder("./dataset/dataset/train", transform=train_transform)
loader =  torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

In [9]:
# источник - https://forums.fast.ai/t/image-normalization-in-pytorch/7534/7

batch_std = []
for image, _ in loader:
  batch_std.append(np.array(image.std(dim=(0,2,3))))

batch_std = np.mean(batch_std, axis=0)

In [10]:
batch_std

array([0.2763075 , 0.26846278, 0.28131756], dtype=float32)

лол ладно возможно я была неправа, матстат для умных

In [11]:
TRANSFORM_IMG = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.4802, 0.4481, 0.3976],
    #                     std=[0.2768182 , 0.26890162, 0.28188124] )
    ])

In [12]:
TRAIN_DATA_PATH = "./dataset/dataset/train"
TEST_DATA_PATH = "./dataset/dataset/val"
BATCH_SIZE = 64

In [13]:
train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=TRANSFORM_IMG)
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,  num_workers=2)
test_data = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform=TRANSFORM_IMG)
test_data_loader  = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=2) 

In [14]:
# Just very simple sanity checks
assert isinstance(train_dataset[0], tuple)
assert len(train_dataset[0]) == 2
assert isinstance(train_dataset[1][1], int)
print("tests passed")

tests passed


### Вспомогательные функции, реализация модели

In [15]:
def train_one_epoch(model, train_dataloader, criterion, optimizer, device="cuda:0"):
    model.train()
    train_epoch_loss, train_epoch_true_hits = torch.empty(0).to(device ="cuda:0" ), torch.empty(0).to(device ="cuda:0" )#.to(device), torch.empty(0).to(device)
    
    for imgs, labels in tqdm(train_dataloader):

        imgs, labels = imgs.to(device), labels.to(device)
        y_pred = model(imgs)
        loss = criterion (y_pred, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        #log loss
        train_epoch_loss = torch.cat((train_epoch_loss, loss.unsqueeze(0) / labels.size(0)))
        train_loss_log = loss.data / labels.size(0)

        #log accuracy
        pred_classes = torch.argmax(y_pred, dim=-1)
        train_epoch_true_hits = torch.cat((
            train_epoch_true_hits, 
            (pred_classes == labels).sum().unsqueeze(0)
        ))

        train_acc_log = (pred_classes == labels).cpu().sum() / labels.shape[0]

    # logging progress, graphs etc
    print("Train loss (mean):", train_epoch_loss.mean().item())
    print(
        "Train acc:", 
        (train_epoch_true_hits.sum() / train_epoch_true_hits.size(0) / train_dataloader.batch_size).item()
        )
    wandb.log({"Train loss (mean)": (train_epoch_loss.mean().item()), "Train acc":(train_epoch_true_hits.sum() / train_epoch_true_hits.size(0) / train_dataloader.batch_size).item()})

def predict(model, val_dataloader, criterion, device="cuda:0"):

    model.to(device)
    
    losses = []
    # когда я через тензоры пытаюсь сделать выплывает ошибка RuntimeError: CUDA out of memory мне не нравится так что нумпи
    # я знаю можно было бы долбаться с торчем но мне так лень если честно

    predicted_classes = np.array([])
    true_classes = np.array([])

    model.eval()
    torch.no_grad() #да у меня до этого еще есть no grad просто меня так смущало то что acc лучше на тесте что я перестраховалась
    for imgs, labels in val_dataloader:
      imgs, labels = imgs.to(device), labels.to(device)
      y_pred = model(imgs)
      loss = criterion(y_pred, labels)
      
      predicted_classes = np.append(predicted_classes, y_pred.argmax(1).cpu().detach().numpy())
      true_classes = np.append(true_classes, labels.cpu().detach().numpy())
      losses.append(loss.detach().item())

    return losses, predicted_classes, true_classes


def train(model, train_dataloader, val_dataloader, criterion, optimizer, device="cuda:0", n_epochs=10, scheduler=None):
    model.to(device)
    for epoch in range(n_epochs):

      # thats where we train
      print('epoch ', epoch)
      train_one_epoch(model, train_dataloader, criterion, optimizer)

      #thats where we predict
      with torch.no_grad():
        losses, predicted_classes, true_classes = predict(model, val_dataloader, criterion)
      
        print("Val loss (total):", np.array(losses).mean())
        print(
            "Val acc:", accuracy_score(true_classes, predicted_classes)
        )
        wandb.log({"Val loss (total)": (np.array(losses).mean()), "Train acc": (accuracy_score(true_classes, predicted_classes))})

In [16]:
device = torch.device("cuda:0")

In [ ]:
all_losses, predicted_labels, true_labels = predict(model, test_data_loader, criterion, device)


In [ ]:
np.array(all_losses).mean()

5.315492900313845

In [ ]:
accuracy_score(predicted_labels, true_labels)

0.0058

In [ ]:
predicted_labels

array([  0.,   0.,  94., ..., 199., 199., 199.])

In [ ]:
len(true_labels)

16

### Обучение модели, запуски экспериментов

там все важное в экспериментах, я бы их сперва запустила! соре что нелинейно!

Простой тест на проверку правильности написанного кода

In [38]:
all_losses, predicted_labels, true_labels = predict(model, test_data_loader, criterion, device)
# assert len(predicted_labels) == len(test_data_loader)
accuracy = accuracy_score(predicted_labels, true_labels)
print("tests passed")

tests passed


In [39]:
print("Оценка за это задание составит {} баллов".format(min(10, 10 * accuracy / 0.44)))

Оценка за это задание составит 6.656818181818181 баллов


In [40]:
len(predicted_labels)

10000

In [41]:
len(test_data_loader)

157

я если честно не совсем понимаю как они должны приравниваться ассертом, но len(predicted_labels) у меня равен количеству объектов в тесте (типа 200 папок по 50 объектов), так что я думаю все норм

просто окей подумайте сами если они равны то len(predicted labels) должно быть равно 157. но поскольку загоняем мы туда весь валидационный даталоадер то лейблов у нас должно быть столько же, сколько элементов в нашей валидационной выборке, а их 10000. иначе мы просто не юзаем в функции весь переданный материал и это либо 1) неоптимально (?) потому что функцию приходится перезапускать каждый раз передавая в нее кучу всего 2) неправильно, если мы реально потом accuracy пытаемся оценить по 157 элементам из 10000. так что да, мой код ассерт не прошел, я читала указание сверху и готова к каре, но это не значит, что код неправильный, с ним-то все ок.

In [ ]:
def predict_2(model, imgs, labels, criterion, device="cuda:0"):
    y_pred = model(imgs)
    loss = criterion(y_pred, labels)
    
    predicted_classes = y_pred.argmax(1).cpu().detach().numpy()
    true_classes = true_classes, labels.cpu().detach().numpy()
    losses = loss.detach().item()

    return losses, predicted_classes, true_classes


def train_2(model, train_dataloader, val_dataloader, criterion, optimizer, device="cuda:0", n_epochs=1, scheduler=None):
    model.to(device)
    for epoch in range(n_epochs):

      # thats where we train
      print('epoch ', epoch)
      train_one_epoch(model, train_dataloader, criterion, optimizer)

      #thats where we predict
      losses = []
      predicted_classes = np.array([])
      true_classes = np.array([])

      with torch.no_grad():
        for imgs, labels in val_dataloader:
          imgs, labels = imgs.to(device), labels.to(device)
          l, p, t = predict(model, imgs, labels, criterion)
          predicted_classes = np.append(predicted_classes, p)
          true_classes = np.append(true_classes, t)
          losses.append(l)
        print("Val loss (total):", np.array(losses).mean())
        print(
            "Val acc:", accuracy_score(true_classes, predicted_classes)
        )
        wandb.log({"Val loss (total)": (np.array(losses).mean()), "Train acc": (accuracy_score(true_classes, predicted_classes))})

Запустить обучение можно в ячейке ниже.

### Проверка полученной accuracy

После всех экспериментов которые вы проделали, выберите лучшую из своих моделей, реализуйте и запустите функцию `evaluate`. Эта функция должна брать на вход модель и даталоадер с валидационными данными и возврашать accuracy, посчитанную на этом датасете.

In [42]:
def evaluate (model, val_dataloader):
  criterion = nn.CrossEntropyLoss()
  _, pred, true = predict(model, val_dataloader, criterion)
  return accuracy_score(true, pred)

In [44]:
accuracy = evaluate(model, test_data_loader)

In [45]:
print("Оценка за это задание составит {} баллов".format(min(10, 10 * accuracy / 0.44)))

Оценка за это задание составит 6.656818181818181 баллов


### Отчёт об экспериментах 

текст писать тут

сначала попробуем пример из семинаров, это как будто самый простой и логичный вариант для первичной отладки. только изменим количество выходов!

In [18]:
class ModelBaseline(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 5, padding = 1)
        self.conv2 = nn.Conv2d(16, 16, 5, padding = 2)
        self.linear1 = nn.Linear(15376, 1000)
        self.linear2 = nn.Linear(1000, 200)
        self.pool = nn.MaxPool2d(2)
        self.softmax = nn.Softmax(dim = 0)
        self.flatten = nn.Flatten()

    def forward(self, x):
        # x: (batch_size; n_channels; hight; width)
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.flatten(x)
        x = F.relu(self.linear1(x))
        x = self.softmax(self.linear2(x))

        return x
model = ModelBaseline().to(device)
wandb.watch(model)

In [20]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()
train(model,train_data_loader, test_data_loader, criterion, optimizer)

epoch  0


100%|██████████| 1563/1563 [01:32<00:00, 16.92it/s]


Train loss: 0.08283918350934982
Train acc: 0.0049484167248010635
Val loss: 5.298314537971643
Val acc: 0.0044
epoch  1


100%|██████████| 1563/1563 [01:32<00:00, 16.94it/s]


Train loss: 0.08283906430006027
Train acc: 0.00564819248393178
Val loss: 5.298310935876931
Val acc: 0.005
epoch  2


...

Train loss: 0.08283782005310059
Train acc: 0.014485364779829979


Val loss: 5.298271091121017
Val acc: 0.0104
epoch  9


100%|██████████| 1563/1563 [01:32<00:00, 16.96it/s]

Train loss: 0.08283747732639313
Train acc: 0.015625


Val loss: 5.298259914301004
Val acc: 0.0106


это конечно очень красиво и приятно смотреть как эпоха от эпохи качество на валидации растет, но стоит попробовать что-то чуть более глубокое и умное. например хороший вариант для лентяев это готовые сетки торча. обученные брать в этом задании нельзя - возьму необученные

In [21]:
model = models.resnet18(pretrained=False).to(device)
model.fc = nn.Linear(512, 200)

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [22]:
train(model,train_data_loader, test_data_loader, criterion, optimizer)

epoch  0


100%|██████████| 1563/1563 [02:26<00:00, 10.63it/s]


Train loss: 0.07702969759702682
Train acc: 0.03990722820162773
Val loss: 4.569371504388797
Val acc: 0.0705
epoch  1


100%|██████████| 1563/1563 [02:26<00:00, 10.67it/s]

Train loss: 0.0672219842672348
Train acc: 0.10646592825651169


...

Train loss: 0.03747577220201492
Train acc: 0.4284628927707672


Val loss: 3.560207702551678
Val acc: 0.2232
epoch  9


100%|██████████| 1563/1563 [02:28<00:00, 10.56it/s]

Train loss: 0.03350477293133736
Train acc: 0.48256558179855347


Val loss: 3.5435245750816007
Val acc: 0.2373


тут вообще копаться особо не с чем кроме как с lr и моментумом

In [35]:
def train(model, train_dataloader, val_dataloader, criterion, optimizer, device="cuda:0", n_epochs=8, scheduler=None):
    model.to(device)
    for epoch in range(n_epochs):

      # thats where we train
      print('epoch ', epoch)
      train_one_epoch(model, train_dataloader, criterion, optimizer)

      #thats where we predict
      with torch.no_grad():
        losses, predicted_classes, true_classes = predict(model, val_dataloader, criterion)
      
        print("Val loss:", np.array(losses).mean())
        print(
            "Val acc:", accuracy_score(true_classes, predicted_classes)
        )
        wandb.log({"Val loss": (np.array(losses).mean()), "Train acc": (accuracy_score(true_classes, predicted_classes))})

In [36]:
model = models.resnet18(pretrained=False).to(device)
model.fc = nn.Linear(512, 200)

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.99)

In [37]:
train(model,train_data_loader, test_data_loader, criterion, optimizer)

epoch  0


100%|██████████| 1563/1563 [02:27<00:00, 10.57it/s]


Train loss: 0.07253139466047287
Train acc: 0.06874799728393555
Val loss: 4.274642762105176
Val acc: 0.1055
epoch  1


100%|██████████| 1563/1563 [02:25<00:00, 10.71it/s]


Train loss: 0.05802208557724953
Train acc: 0.1822216957807541
Val loss: 3.5111601610851895
Val acc: 0.2117
epoch  2


...

Train loss: 0.034818749874830246
Train acc: 0.44669705629348755
Val loss: 3.155269601542479
Val acc: 0.2924
epoch  6


100%|██████████| 1563/1563 [02:25<00:00, 10.71it/s]


Train loss: 0.02914540097117424
Train acc: 0.5227727293968201
Val loss: 3.3817776031554883
Val acc: 0.2823
epoch  7


100%|██████████| 1563/1563 [02:26<00:00, 10.67it/s]


Train loss: 0.02313731238245964
Train acc: 0.6079554557800293
Val loss: 3.444031590868713
Val acc: 0.2929
